# Visual Covariance Test

$$
% latex macros.  this does not work when converting to latex.
\newcommand{\bm}[1]{\boldsymbol{#1}}
\DeclareMathOperator{\cov}{cov}
\DeclareMathOperator{\diag}{diag}
\newcommand{\ud}{\mathrm{d}}
\newcommand{\XX}{{\bm X}}
\newcommand{\VV}{{\bm V}}
\newcommand{\N}{\mathcal{N}}
\newcommand{\mmu}{{\bm \mu}}
\newcommand{\SSi}{{\bm \Sigma}}
\newcommand{\ssi}{{\bm \sigma}}
$$

Pick a covariance type. Then for $x_t$ and $v_t = \ud x_t/\ud t$, visually check that
$$
\begin{aligned}
\cov(x_t, v_s) & = \int_0^t \cov(v_u, v_s) \, \ud u \\
\cov(x_t, x_s) & = \int_0^t\int_0^s \cov(v_u, v_w) \, \ud u\, \ud w
\end{aligned}
$$
by comparing analytic formula to numerical integration with `scipy.integrate.quad` and `scipy.integrate.dblquad`.

In [1]:
import os
#os.chdir('C:\\Users\\mohan\\Documents\\probDE')
os.chdir('../')

In [2]:
import numpy as np
from numpy.random import uniform as runif
from scipy import integrate
import BayesODE as bo
from Tests.ode_bayes_star import ode_bayes_star

In [ ]:
npts = 50
# pick covariance type
type = "ex"
if type == "se":
    cov_vv = bo.cov_vv_se
    cov_xv = bo.cov_xv_se
    cov_xx = bo.cov_xx_se
elif type == "ex":
    cov_vv = bo.cov_vv_ex
    cov_xv = bo.cov_xv_ex
    cov_xx = bo.cov_xx_ex
elif type == "re":
    cov_vv = bo.cov_vv_re
    cov_xv = bo.cov_xv_re
    cov_xx = bo.cov_xx_re
else:
    raise ValueError("Invalid covariance type.")
# plot cov_vv
tmax = runif(low = 0, high = 10, size = 1)
gamma = runif(0, tmax/10, 1)
tseq = np.linspace(start = 0, stop = tmax, num = npts)
av = np.zeros((npts, 1))
for ii in range(0,npts):
    av[ii] = cov_vv(0, tseq[ii], gamma)
plt.rcParams['figure.figsize'] = [10, 6]
plt.subplot(3,2,(1,2))
plt.plot(tseq,av)
# integrate once and compare to analytic solution
t = runif(0, tmax, 1) # first time point
s = runif(0, tmax, 1) # second time point
tseq = np.linspace(start = 0, stop = tmax, num = npts)
xv_an = np.zeros((npts,1)) # analytic solution
xv_nu = xv_an # numeric solution
for ii in range(0,npts):
    xv_an[ii] = cov_xv(tseq[ii], s, gamma)
    xv_nu[ii],err = integrate.quad(cov_vv, 0, tseq[ii], args=(s,gamma))
plt.subplot(3,2,3)
plt.plot(tseq,xv_an,'r',tseq,xv_nu,'b')
plt.subplot(3,2,4)
plt.plot(tseq,abs(xv_an-xv_nu))
# integrate twice and compare to analytic solution
xx_an = np.zeros((npts,1)) # analytic solution
xx_nu = xx_an # numeric solution
for ii in range(0,npts):
    xx_an[ii] = cov_xx(tseq[ii], s, gamma)
    xx_nu[ii],err = integrate.dblquad(cov_vv, 0, tseq[ii], 0, s, args = (gamma))
plt.subplot(3,2,5)
plt.plot(tseq,xx_an,'r',tseq,xx_nu,'b')
plt.subplot(3,2,6)
plt.plot(tseq,abs(xx_an-xx_nu))

# Updating Algorithm Test

Let $T = (t_1, \ldots, t_N)$ denote the evaluation time points, and $\XX$ and $\VV$ denote vectors of length $N$ corresponding to $x(T)$ and $v(T)$.  The solution prior is then expressed as
$$
\begin{bmatrix} \XX \\ \VV \end{bmatrix} \sim \N\left(\begin{bmatrix} \mmu_X \\ \mmu_V \end{bmatrix}, \begin{bmatrix} \SSi_{XX} & \SSi_{XV} \\ \SSi_{VX} & \SSi_{VV} \end{bmatrix} \right).
$$
Now suppose that the $N$ model interrogations $\VV_\star$ and their predictive variances $\ssi_\star^2$ are given in advance.  Then joint distribution of $(\XX, \VV, \VV_\star)$ is
$$
\begin{bmatrix} \XX \\ \VV \\ \VV_\star \end{bmatrix} \sim \N\left(\begin{bmatrix} \mmu_X \\ \mmu_V \\ \mmu_V \end{bmatrix}, \begin{bmatrix} \SSi_{XX} & \SSi_{XV} & \SSi_{XV} \\ \SSi_{VX} & \SSi_{VV} & \SSi_{VV} \\ \SSi_{VX} & \SSi_{VV} & \SSi_{VV} + \diag(\ssi_\star^2) \end{bmatrix} \right),
$$
from which we can deduce calculate the mean and variance of the final update (normal) distribution, $p(\XX \mid \VV_\star)$.  This suggests the following unit test for the ODE solver:

1. Modify `ode_bayes` such that it accepts an optional input of $\VV_\star$ and $\ssi_\star^2$ (please use argument names consistent with the rest of the function).
2. Write a function to compute the mean and variance of $p(\XX \mid \VV_\star)$ based on $(\mmu_X, \mmu_V, \SSi_{XX}, \SSi_{XV}, \SSi_{VV}, \VV_\star)$.
3. Run `ode_bayes` with predetermined $(\VV_\star, \ssi_\star^2)$, and check that the mean and variance output is identical to that of step 2.

In [71]:
#Toy example
def f(x,t):
    return  3*(t+1/4) - x/(t+1/4)

a = 0
b = 4.75
x0 = 0
N = 100
tseq = np.linspace(a, b, N)
w = gamma = 1.0642211055276383
alpha = N/10

mu_v = np.zeros((N)) # prior mean of v(tseq)
mu_x = x0 + mu_v # prior mean of x(tseq)

Sigma_vv = bo.cov_vv_ex(tseq, tseq, gamma, alpha)
Sigma_xx = bo.cov_xx_ex(tseq, tseq, gamma, alpha)
Sigma_xv = bo.cov_xv_ex(tseq, tseq, gamma, alpha)

In [6]:
x = integrate.odeint(f,x0,tseq)
x = np.ndarray.flatten(x)
v = np.array([f(xl,tseq[i]) for i, xl in enumerate(x)])
sigma_star_avg = (max(v) - min(v))*0.05

upper = 3*sigma_star_avg/2
lower = sigma_star_avg/2

sigma_star = runif(lower, upper)
vstar = v+sigma_star*np.random.normal(0,1)

In [7]:
mu_star, Sigma_star = ode_bayes_star(mu_x, mu_v, Sigma_vv, Sigma_xx, Sigma_xv, vstar)

In [8]:
_, mu_x, Sigma_xx = bo.ode_bayes(f, tseq, x0, Sigma_vv, Sigma_xx, Sigma_xv, vstar)

In [9]:
np.allclose(mu_star, mu_x), np.allclose(Sigma_star, Sigma_xx)

(True, True)

## Implementation via the Kalman Filter

For the special case of an exponential kernel (or equivalently, exponential autocorrelation of the solution derivative), the Bayesian ODE solver can be efficiently implemented via Kalman filtering and smoothing algorithms.  To see this, the first step is to derive the conditional distribution $p(y_t \mid y_s)$, where $y_t = (x_t, v_t)$ and $0 < s < t$.

In [46]:
import scipy.linalg as scl

def mvCond(mu, Sigma, icond):
    """
    For y ~ N(mu, Sigma), returns A, b, and V such that y[~icond] | y[icond] ~ N(A * y[icond] + b, V).
    """
    # if y1 = y[~icond] and y2 = y[icond], should have A = Sigma12 * Sigma22^{-1}
    A = np.dot(Sigma[np.ix_(~icond, icond)],scl.cho_solve(scl.cho_factor(Sigma[np.ix_(icond,icond)]), np.identity(sum(icond))))
    b = mu[~icond] - np.dot(A, mu[icond]) # mu1 - A * mu2
    V = Sigma[np.ix_(~icond,~icond)] - np.dot(A, Sigma[np.ix_(icond,~icond)]) # Sigma11 - A * Sigma21
    return A, b, V

# simple test
mu = np.array([1, 2, 3, 4, 5])
Sigma = np.diag([1, 2, 3, 4, 5])
icond = np.array([True, True, True, False, False])
i,o,p = mvCond(mu, Sigma, icond)


array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [10]:
import scipy.stats as scs

ntest = 3
alpha, gamma = 1, 1

for ii in range(ntest):
    nT = scs.randint.rvs(2, 10, size = 1)
    tseq = abs(scs.norm.rvs(size = nT))
    Sig_xx = bo.cov_xx_ex(tseq, tseq, gamma, alpha)
    Sig_xv = bo.cov_xv_ex(tseq, tseq, gamma, alpha)
    Sig_vv = bo.cov_vv_ex(tseq, tseq, gamma, alpha)
    Sigma = np.vstack((np.hstack((Sig_xx, Sig_xv)),
                                    np.hstack((np.transpose(Sig_xv), Sig_vv))))
    
    if any(i<0 for i in np.linalg.eigvalsh(Sigma)):
        print('failed')
        print(Sigma)

In [11]:
import scipy.stats as scs

ntest = 3
alpha, gamma = 1, 1

for ii in range(ntest):
    nT = scs.randint.rvs(2, 10, size = 1)
    tseq = abs(scs.norm.rvs(size = nT))
    Sig_xx = bo.cov_xx_re(tseq, tseq, gamma, alpha)
    Sig_xv = bo.cov_xv_re(tseq, tseq, gamma, alpha)
    Sig_vv = bo.cov_vv_re(tseq, tseq, gamma, alpha)
    Sigma = np.vstack((np.hstack((Sig_xx, Sig_xv)),
                                    np.hstack((np.transpose(Sig_xv), Sig_vv))))
    
    if any(i<0 for i in np.linalg.eigvalsh(Sigma)):
        print('failed')
        print(Sigma)

In [37]:
import scipy.stats as scs

ntest = 3
alpha, gamma = 1, 1

for ii in range(ntest):
    nT = scs.randint.rvs(2, 10, size = 1)
    tseq = abs(scs.norm.rvs(size = nT))
    Sig_xx = bo.cov_xx_se(tseq, tseq, gamma, alpha)
    Sig_xv = bo.cov_xv_se(tseq, tseq, gamma, alpha)
    Sig_vv = bo.cov_vv_se(tseq, tseq, gamma, alpha)
    Sigma = np.vstack((np.hstack((Sig_xx, Sig_xv)),
                                    np.hstack((np.transpose(Sig_xv), Sig_vv))))
    
    if any(round(i, 10) <0 for i in np.linalg.eigvalsh(Sigma)):
        print('failed')
        print(Sigma)

In [38]:
from pykalman import KalmanFilter

In [85]:
tseq2 = np.linspace(a,b,50)
def mvCond(tseq, x0):
    N = len(tseq)
    mu = [x0, 0]*N
    Sigma = np.matrix([[0]*N*2]*N*2)
    Sigma_xx = bo.cov_xx_ex(tseq, tseq, gamma, 1)
    Sigma_xv = bo.cov_xv_ex(tseq, tseq, gamma, 1)
    Sigma_vv = bo.cov_vv_ex(tseq, tseq, gamma, 1)
    
    for i in range(N):
        for j in range(N):
            Sigma[2*i, 2*j] = Sigma_xx[i, j]
            Sigma[2*i, 2*j+1] = Sigma_xv[i, j]
            Sigma[2*i+1, 2*j] = Sigma_xv[j, i]
            Sigma[2*i+1, 2*j+1] = Sigma_vv[i, j]
            
    Sigma_00 = Sigma[0:2*N-2, 0:2*N-2]
    Sigma_10 = Sigma[2*N-2:, 0:2*N-2]
    Sigma_01 = Sigma[0:2*N-2, 2*N-2:]
    Sigma_11 = Sigma[2*N-2:, 2*N-2:]
    A = Sigma_10*np.linalg.pinv(Sigma_00)
    b = mu[2*N-2:]
    V = Sigma_11 - Sigma_10*np.linalg.pinv(Sigma_00)*Sigma_01
    return A,b,V
A, b, V = mvCond(tseq2, x0)
A

matrix([[  0.00000000e+00,  -2.08811917e-14,  -4.15954860e-11,
           6.75675676e-01,   1.22565647e-12,   2.70270270e-01,
           2.41226425e-11,   3.33066907e-15,  -2.34613306e-11,
          -4.32432432e-01,  -1.15315530e-12,  -1.08108108e-01,
          -6.37850250e-12,  -1.08108108e-01,  -1.74775032e-11,
           1.14352972e-13,   1.91891892e+01,  -1.08108108e-01,
           3.40540541e+00,  -1.08108108e-01,  -1.83783784e+00,
          -1.08108108e-01,  -3.48648649e+00,  -4.32432432e-01,
          -3.26161624e-14,  -4.32432432e-01,  -1.77795062e-14,
           2.30926389e-13,  -1.08108108e-01,  -2.27706742e-13,
          -3.24324324e-01,   6.62162162e+00,   4.32432432e-01,
           2.70270270e-01,   6.62162162e+00,   2.70270270e-01,
          -6.35135135e+00,  -5.40540541e-01,  -8.10810811e-01,
          -1.12021503e-13,   5.40540541e-01,  -7.16216216e+00,
          -7.16216216e+00,   6.75675676e-01,   7.83783784e+00,
           6.75675676e-01,  -6.00000000e+00,  -5.324324

In [58]:
Sigma_vv = bo.cov_vv_ex(tseq, tseq, gamma, 1)
Sigma_xx = bo.cov_xx_ex(tseq, tseq, gamma, 1)
Sigma_xv = bo.cov_xv_ex(tseq, tseq, gamma, 1)

Sigma_00 = np.matrix([[Sigma_xx[0,0], Sigma_xv[0,0]], [Sigma_xv[0,0], Sigma_vv[0,0]]]) #cov(x_0, v_0)
Sigma_10 = np.matrix([[Sigma_xx[1,0], Sigma_xv[1,0]], [Sigma_xv[1,0], Sigma_vv[1,0]]]) #cov(x_1, v_0)
A = Sigma_10*np.linalg.pinv(Sigma_00)

Sigma_11 = np.matrix([[Sigma_xx[1,1], Sigma_xv[1,1]], [Sigma_xv[1,1], Sigma_vv[1,1]]]) #cov(x_1, v_1)
Sigma_01 = np.matrix([[Sigma_xx[0,1], Sigma_xv[0,1]], [Sigma_xv[0,1], Sigma_vv[0,1]]]) #cov(x_0, v_1)

CC = Sigma_11 - Sigma_10*np.linalg.pinv(Sigma_00)*Sigma_01

In [82]:
A

matrix([[ 0.00729933,  0.08302055],
        [ 0.08302055,  1.        ]])

In [83]:
CC

matrix([[ 0.02236403,  0.01913056],
        [-0.51456275,  1.13586553]])

In [72]:
def bayes_ode_kalman(fun, x0, tseq, gamma, A, CC):
    N = len(tseq)
    y0 = [x0, f(x0, tseq[0])]
    mu = [y0]*N
    Sigma = [[[0, 0], [0,0]]]*N
    yt = np.array([y0]*N)
    mu_nn = np.array((A*np.matrix(mu[0]).T)).T
    Sigma_nn = A*Sigma[0]*A.T + CC
    kf = KalmanFilter(initial_state_mean=mu[0], initial_state_covariance=Sigma[0], n_dim_obs=2) #Initialization of Kalman Filter
    for n in range(N-1):
        mu_nn = np.array((A*np.matrix(mu[n]).T)).T
        Sigma_nn = A*Sigma[n]*A.T + CC
        
        #Model Interrogation Step
        xt = np.random.normal(mu_nn[0][0], Sigma_nn[0,0])
        vt = fun(xt, tseq[n+1])
        yt[n+1] = [xt,vt]
        mu[n+1], Sigma[n+1] = kf.filter_update(mu_nn[0], Sigma_nn, yt[n+1])
    
    kf2 = KalmanFilter(initial_state_mean=mu[0], initial_state_covariance=Sigma[0], n_dim_obs=2)
    yt_mean = kf2.smooth(yt)[0]
    
    return mu, yt_mean

In [73]:
u,v = bayes_ode_kalman(f, x0, tseq, gamma, A, CC)


In [75]:
u

[[0, 0.75], masked_array(data = [-0.00243948  0.81167695],
              mask = False,
        fill_value = 1e+20), masked_array(data = [-0.00839369  0.96529588],
              mask = False,
        fill_value = 1e+20), masked_array(data = [-0.05328399  1.2824992 ],
              mask = False,
        fill_value = 1e+20), masked_array(data = [-0.02683804  1.34882488],
              mask = False,
        fill_value = 1e+20), masked_array(data = [-0.01637294  1.43616911],
              mask = False,
        fill_value = 1e+20), masked_array(data = [-0.0754925   1.73772582],
              mask = False,
        fill_value = 1e+20), masked_array(data = [-0.08236826  1.9208504 ],
              mask = False,
        fill_value = 1e+20), masked_array(data = [-0.06749333  2.02250985],
              mask = False,
        fill_value = 1e+20), masked_array(data = [-0.08905396  2.19800076],
              mask = False,
        fill_value = 1e+20), masked_array(data = [-0.07941817  2.31844051],
     